Import Libraries

In [1]:
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
%matplotlib inline
np.random.seed(123)

Using TensorFlow backend.


In [2]:
import os
from PIL import Image, ImageOps

Creating mirrored images for class imbalance

In [ ]:
for filename in os.listdir('chest_xray/train/NORMAL'):
    if filename.endswith('.jpeg'):
        im = Image.open(os.path.join('chest_xray/train/NORMAL/', filename))
        im_mirror = ImageOps.mirror(im)
        im_mirror.save('whatamidoing/mirror_'+filename)
#         print(os.path.join('chest_xray/train/NORMAL/', filename))
    else:
        continue

In [3]:
train = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        directory = 'chest_xray/train',
        class_mode = 'binary',
        shuffle=False,
        batch_size = 6558)

Found 6558 images belonging to 2 classes.


In [4]:
test = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        directory = 'chest_xray/test',
        class_mode = 'binary',
        shuffle=False,
        batch_size = 624)

Found 624 images belonging to 2 classes.


In [6]:

# Spliting images and labels
# val_images, val_labels = next(val)
test_images, test_labels = next(test)
train_images, train_labels = next(train)

In [7]:
from sklearn.model_selection import train_test_split
X_model, X_test, y_model, y_test = train_test_split(train_images, train_labels, test_size=0.20, random_state=123)

In [8]:
y_model = np.reshape(y_model, (len(y_model),1))
y_test = np.reshape(y_test, (len(y_test),1))
test_labels = np.reshape(test_labels, (len(test_labels),1))

In [9]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256,  3)))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Conv2D(32, (3, 3), activation='relu'))
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(32, activation='relu'))
cnn.add(layers.Dropout(0.5))
cnn.add(layers.Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [ ]:

cnn1 = cnn.fit(X_model,
                    y_model,
                    epochs=5,
                    batch_size=50,
                    validation_data=(X_test, y_test))